In [1]:
import jupyter_black
from datetime import datetime, timedelta
import pandas
import altair as alt
import numpy
import requests

jupyter_black.load()

MY_STRAVA_CLIENT_ID = 125463
MY_STRAVA_CLIENT_SECRET = "017fafbf5c5067490f1382fd3454c30cdb61f4b0"
DAYS_BACK = 180

In [2]:
def get_token():

    url = f"https://www.strava.com/oauth/authorize?client_id={MY_STRAVA_CLIENT_ID}&redirect_uri=http://127.0.0.1:5000/authorization&approval_prompt=auto&scope=read,activity:read&response_type=code"
    print(f"Go to {url}")
    response = input(f"Enter the full URL:")
    code = response.split("code=")[1].split("&")[0]

    params = {
        "client_id": MY_STRAVA_CLIENT_ID,
        "client_secret": MY_STRAVA_CLIENT_SECRET,
        "code": code,
        "grant_type": "authorization_code",
    }
    print(params)

    response = requests.post("https://www.strava.com/oauth/token", params=params)
    print(response.json())
    token = response.json()["access_token"]

    return token


token = get_token()
print(f"{token=}")

Go to https://www.strava.com/oauth/authorize?client_id=125463&redirect_uri=http://127.0.0.1:5000/authorization&approval_prompt=auto&scope=read,activity:read&response_type=code


{'client_id': 125463, 'client_secret': '017fafbf5c5067490f1382fd3454c30cdb61f4b0', 'code': 'bf8c1341c62007617eaf72bc9cb4b32b24e4c713', 'grant_type': 'authorization_code'}
{'token_type': 'Bearer', 'expires_at': 1720473893, 'expires_in': 16785, 'refresh_token': '4f6bcdaaa996c6d058e6750cd3cc846f8fb8fe19', 'access_token': '6deb123cd27fd8aadeb33ad14f1a691360af4789', 'athlete': {'id': 44717295, 'username': None, 'resource_state': 2, 'firstname': 'Duarte', 'lastname': 'Carmo', 'bio': 'duarteocarmo.com', 'city': 'Copenhagen ', 'state': 'DK', 'country': None, 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2019-07-27T19:20:59Z', 'updated_at': '2024-07-05T16:46:28Z', 'badge_type_id': 0, 'weight': 72.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/44717295/13646951/6/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/44717295/13646951/6/large.jpg', 'friend': None, 'follower': None}}
token='6deb123cd27fd8aadeb33ad14f1a691360af

In [3]:
def fetch_activities(access_token: str) -> list:
    url = f"https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    after_timestamp = int((datetime.now() - timedelta(days=DAYS_BACK)).timestamp())

    per_page = 200
    page = 1
    all_activities = []

    while True:
        params = {"after": after_timestamp, "page": page, "per_page": per_page}
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
        activities = response.json()

        if not activities:
            break

        all_activities.extend(activities)
        if len(activities) < per_page:
            break

        page += 1

    return all_activities


activities = fetch_activities(token)

In [4]:
running_activities = [act for act in activities if act["type"] == "Run"]

In [5]:
data = {
    "start_date": [act["start_date"] for act in running_activities],
    "distance_meters": [float(act["distance"]) for act in running_activities],
    "time_seconds": [act["moving_time"] for act in running_activities],
}


df = pandas.DataFrame(data)
df["start_date"] = pandas.to_datetime(df["start_date"])
df.set_index("start_date", inplace=True)
weekly_data = df.resample("W").sum()
weekly_data["distance_km"] = round(weekly_data["distance_meters"] / 1000, 1)


def get_tanda_value(km_per_week: int, pace_sec_per_km: int) -> float:
    marathon_distance = 42.195
    marathon_pace_sec_per_km = (
        17.1 + 140.0 * numpy.exp(-0.0053 * km_per_week) + 0.55 * pace_sec_per_km
    )
    total_marathon_time_secs = marathon_distance * marathon_pace_sec_per_km
    total_marathon_time_hours = total_marathon_time_secs / 3600
    return total_marathon_time_hours


def get_pace_for_distance(km_per_week: int, total_marathon_time_hours: float) -> float:
    marathon_distance = 42.195
    marathon_pace_sec_per_km = total_marathon_time_hours * 3600 / marathon_distance
    pace_sec_per_km = (
        marathon_pace_sec_per_km - 17.1 - 140.0 * numpy.exp(-0.0053 * km_per_week)
    ) / 0.55
    return pace_sec_per_km


def pretty_marathon_time(total_marathon_time_hours: float) -> str:
    hours = int(total_marathon_time_hours)
    minutes = int((total_marathon_time_hours - hours) * 60)
    seconds = int(((total_marathon_time_hours - hours) * 60 - minutes) * 60)
    if seconds >= 30:
        minutes += 1

    return f"{hours} hours {minutes} minutes"

In [6]:
daily_df = df.groupby(df.index.date).sum()
daily_df.index = pandas.to_datetime(daily_df.index)
daily_df.index.name = "date"


daily_df["tanda_day"] = get_tanda_value(
    daily_df["distance_meters"] / 1000 * 7,
    daily_df["time_seconds"] / (daily_df["distance_meters"] / 1000),
)
daily_df["tanda_day_pretty"] = pandas.to_datetime(daily_df["tanda_day"], unit="h")


daily_df = daily_df.reset_index()
daily_df["date"] = pandas.to_datetime(daily_df["date"])
daily_df.set_index("date", inplace=True)

num_weeks = 8
num_days = num_weeks * 7
rolling = f"{num_days}d"


daily_df["rolling_distance_meters"] = (
    daily_df["distance_meters"].rolling(window=rolling).sum()
)
daily_df["rolling_time_seconds"] = (
    daily_df["time_seconds"].rolling(window=rolling).sum()
)

daily_df["rolling_km_per_week"] = daily_df["rolling_distance_meters"] / 1000 / num_weeks
daily_df["rolling_pace_sec_per_km"] = (
    daily_df["rolling_time_seconds"] / daily_df["rolling_distance_meters"] * 1000
)

daily_df["rolling_tanda_day"] = get_tanda_value(
    daily_df["rolling_km_per_week"], daily_df["rolling_pace_sec_per_km"]
)

daily_df["rolling_tanda_day_pretty"] = pandas.to_datetime(
    daily_df["rolling_tanda_day"], unit="h"
)

daily_df["type_rolling"] = "Tanda (8 weeks)"
daily_df["type_daily"] = "Tanda (daily)"

daily_df["pace_sec_per_km"] = daily_df["time_seconds"] / (
    daily_df["distance_meters"] / 1000
)
daily_df["distance_km"] = daily_df["distance_meters"] / 1000

daily_df = daily_df.sort_values(by="date", ascending=True)
daily_df["date_factor"] = numpy.exp(numpy.linspace(0, 15, len(daily_df)))


daily_df["daily_pace_pretty"] = daily_df["pace_sec_per_km"].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d}"
)
daily_df["rolling_pace_pretty"] = daily_df["rolling_pace_sec_per_km"].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d}"
)

daily_df["rolling_km_per_week_daily_distance"] = daily_df["rolling_km_per_week"] / 7
daily_df["Latest run"] = "Latest run"

daily_df["pretty_rolling_tanda_day"] = daily_df["rolling_tanda_day"].apply(
    pretty_marathon_time
)

# current_form_marathon_time = pretty_marathon_time(
#     daily_df.loc[start_date:last_date]
#     .reset_index()
#     .sort_values("date")
#     .tail(1)["rolling_tanda_day"]
#     .item()
# )


def pace_tick_formatter(value):
    minutes = int(value // 60)
    seconds = int(value % 60)
    return f"{minutes}:{seconds:02d}"

In [7]:
x_scale = alt.Scale(padding=20)
upper_limit = weekly_data["distance_km"].max()
lower_limit = 0

x = alt.X("start_date:T", scale=alt.Scale(padding=20), title="Week")
y = alt.Y(
    "distance_km:Q",
    axis=alt.Axis(title="Kilometers"),
    scale=alt.Scale(domain=[lower_limit, upper_limit]),
)

line_chart = (
    alt.Chart(weekly_data.reset_index())
    .mark_area(
        line={"color": "#ff561b"},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color="#ff561b", offset=1),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        x=x,
        y=y,
        tooltip=["start_date:T", "distance_km:Q"],
    )
    .properties(width=800, height=500, title="Running distance per week (km)")
)

points = (
    alt.Chart(weekly_data.reset_index())
    .mark_point(
        filled=True,
        fill="white",
        stroke="#ff561b",
        strokeWidth=2,
        size=50,
        shape="circle",
    )
    .encode(
        x=x,
        y=y,
        tooltip=["start_date:T", "distance_km:Q"],
    )
)


chart = line_chart + points

chart.show()

alt.LayerChart(...)

In [8]:
x = alt.X("date:T", title="Date", scale=alt.Scale(padding=20))

daily_line = (
    alt.Chart(daily_df.reset_index())
    .mark_point(shape="square", filled=True, opacity=0.5)
    .encode(
        x=x,
        y=alt.Y("hoursminutes(tanda_day_pretty):O", title="Tanda day"),
        color=alt.value("#d65de0"),
        tooltip=[
            alt.Tooltip("tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)
rolling_line = (
    alt.Chart(daily_df.reset_index())
    .mark_line(interpolate="basis")
    .encode(
        x=x,
        y=alt.Y(
            "hoursminutes(rolling_tanda_day_pretty):O", title="Tanda trend (8 weeks)"
        ),
        color=alt.value("#d65de0"),
        tooltip=[
            alt.Tooltip("rolling_tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)


(daily_line + rolling_line).properties(width=800, height=500, title="Tanda").show()

alt.LayerChart(...)

In [9]:
pace_ticks_values = list(range(240, 60 * 8, 15))
last_date = max(daily_df.index)
start_date = last_date - timedelta(days=56)


daily_df["shape"] = daily_df.index.to_series().apply(
    lambda x: "square" if x == last_date else "circle"
)


daily_line = (
    alt.Chart(daily_df.loc[start_date:last_date].reset_index().sort_values("date"))
    .mark_point(
        filled=True,
        # shape="triangle",
        size=90,
    )
    .encode(
        x=alt.X(
            "distance_km:Q",
            title="Daily Distance (km)",
            axis=alt.Axis(tickCount=int(25 // 5)),
        ),
        y=alt.Y(
            "pace_sec_per_km:Q",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            title="Pace (mm:ss)",
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        tooltip=[
            alt.Tooltip("distance_km:Q", title="Distance (km)", format=".1f"),
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("daily_pace_pretty:N", title="Pace (mm:ss/km)"),
        ],
        color=alt.Color(
            "date_factor:Q", scale=alt.Scale(scheme="lightgreyred"), legend=None
        ),
        shape=alt.Shape(
            "shape:N", scale=alt.Scale(range=["square", "circle"]), legend=None
        ),
    )
)


(daily_line).properties(
    width=800, height=500, title="Pace and daily distance"
).interactive().show()

alt.Chart(...)

In [10]:
marathon_times = []


for marathon_time in numpy.arange(2.5, 4.5, 0.25):
    for km_day in range(0, 50, 1):
        km_week = km_day * 7
        pace = get_pace_for_distance(km_week, marathon_time)
        formatted_pace = pace_tick_formatter(pace)
        marathon_times.append(
            {
                "marathon_time": marathon_time,
                "km_day": km_day,
                "km_week": km_week,
                "pace": pace,
                "formatted_pace": formatted_pace,
            }
        )

times_df = pandas.DataFrame(marathon_times)

In [11]:
daily_df.loc[start_date:last_date].distance_km.max()

16.6094

In [12]:
marathon_times = (
    alt.Chart(times_df)
    .mark_line(interpolate="basis")
    .encode(
        x=alt.X(
            "km_day:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(
                domain=[0, daily_df.loc[start_date:last_date].distance_km.max()]
            ),
        ),
        y=alt.Y(
            "pace:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(300, 405),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        color=alt.Color(
            "marathon_time:N",
            title="Marathon Time",
            scale=alt.Scale(scheme="turbo"),
            legend=alt.Legend(
                labelExpr="floor(datum.value) + ':' + (floor((datum.value % 1) * 60) < 10 ? '0' : '') + floor((datum.value % 1) * 60)"
            ),
        ),
        tooltip=[
            alt.Tooltip("km_day:Q", title="Distance (km)"),
            alt.Tooltip("formatted_pace:N", title="Pace (mm:ss)"),
            alt.Tooltip("marathon_time:Q", title="Marathon time (hours)"),
        ],
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)

marathon_times.show()

alt.Chart(...)

In [13]:
daily_df["Legend"] = "Tanda Progression line"


tooltip = [
    alt.Tooltip(
        "rolling_km_per_week_daily_distance:Q",
        title="Distance (km)",
        format=".1f",
    ),
    alt.Tooltip(
        "rolling_pace_pretty:N",
        title="Pace (s/km)",
    ),
    alt.Tooltip("date:T", title="Date"),
    alt.Tooltip("pretty_rolling_tanda_day:N", title="Marathon Form"),
]


tanda_progression = (
    alt.Chart(daily_df.loc[start_date:last_date].reset_index().sort_values("date"))
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            "rolling_km_per_week_daily_distance:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(zero=False),
        ),
        y=alt.Y(
            "rolling_pace_sec_per_km:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        tooltip=tooltip,
        order="date",
        color=alt.Color(
            "Legend:N",
            legend=alt.Legend(title=None),
            scale=alt.Scale(domain=["Tanda Progression line"], range=["#87f94c"]),
        ),
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)


current_form = (
    alt.Chart(
        daily_df.loc[start_date:last_date].reset_index().sort_values("date").tail(1)
    )
    .mark_point(filled=True, size=70)
    .encode(
        x=alt.X(
            "rolling_km_per_week_daily_distance:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(zero=False),
        ),
        y=alt.Y(
            "rolling_pace_sec_per_km:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        color=alt.value("#142ef5"),
        tooltip=tooltip,
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)

(tanda_progression + current_form).show()

alt.LayerChart(...)

In [14]:
(marathon_times + daily_line + tanda_progression + current_form).properties(
    title="Tanda and marathon pace",
).interactive().show()

alt.LayerChart(...)

In [15]:
x, y = numpy.meshgrid(range(-5, 5), range(-5, 5))
z = x**2 + y**2

# Convert this grid to columnar data expected by Altair
source = pandas.DataFrame({"x": x.ravel(), "y": y.ravel(), "z": z.ravel()})

alt.Chart(source).mark_rect().encode(x="x:O", y="y:O", color="z:Q")

alt.Chart(...)

In [71]:
daily_df["week_number"] = daily_df.index.isocalendar().week
daily_df["day_of_the_week_num"] = daily_df.index.dayofweek
all_dates = pandas.date_range(
    start=daily_df.index.min(), end=daily_df.index.max(), freq="D"
)

heatmap_data = daily_df[
    [
        "week_number",
        "day_of_the_week_num",
        "distance_km",
    ]
].sort_values(["week_number", "day_of_the_week_num"])

heatmap_data = heatmap_data.reindex(all_dates, fill_value=0).reset_index()


alt.Chart(heatmap_data).mark_rect(cornerRadius=4).encode(
    x=alt.X("week_number:O", axis=alt.Axis(title=None)),
    y=alt.Y("day_of_the_week_num:O", axis=alt.Axis(title=None)),
    color=alt.Color("distance_km:Q", scale=alt.Scale(scheme="greens"), legend=None),
).configure_scale(bandPaddingInner=0.2).properties(
    title="Weekly Distance Heatmap"
).interactive().show()

alt.Chart(...)

In [66]:
heatmap_data

,week_number,day_of_the_week_num,distance_km
2024-01-12,2,4,5.9718
2024-01-13,2,5,8.0055
2024-01-14,2,6,7.3164
2024-01-15,3,0,5.0606
2024-01-16,3,1,5.0500
...,...,...,...
2024-07-03,27,2,6.9957
2024-07-04,27,3,6.9167
2024-07-05,27,4,5.0366
2024-07-06,0,0,0.0000


In [62]:
heatmap_data

,date,week_number,day_of_the_week_num,distance_km
0,2024-01-12,2,4.0,5.9718
1,2024-01-13,2,5.0,8.0055
2,2024-01-14,2,6.0,7.3164
3,2024-01-15,3,0.0,5.0606
4,2024-01-16,3,1.0,5.0500
...,...,...,...,...
173,2024-07-03,27,2.0,6.9957
174,2024-07-04,27,3.0,6.9167
175,2024-07-05,27,4.0,5.0366
176,2024-07-06,<NA>,NaN,0.0000


In [27]:
daily_df

,distance_meters,time_seconds,tanda_day,tanda_day_pretty,rolling_distance_meters,rolling_time_seconds,rolling_km_per_week,rolling_pace_sec_per_km,rolling_tanda_day,rolling_tanda_day_pretty,...,date_factor,daily_pace_pretty,rolling_pace_pretty,rolling_km_per_week_daily_distance,Latest run,pretty_rolling_tanda_day,shape,Legend,day_of_week,day_of_week_as_string
date,,,,,,,,,,,,,,,,,,,,,
2024-01-12,5971.8,1920,3.587856,1970-01-01 03:35:16.282006087,5971.8,1920.0,0.746475,321.511102,3.907472,1970-01-01 03:54:26.898042650,...,1.000000e+00,5:21,5:21,0.106639,Latest run,3 hours 54 minutes,circle,Tanda Progression line,4,Friday
2024-01-13,8005.5,2593,3.507717,1970-01-01 03:30:27.782667097,13977.3,4513.0,1.747162,322.880671,3.907655,1970-01-01 03:54:27.558064717,...,1.138043e+00,5:23,5:22,0.249595,Latest run,3 hours 54 minutes,circle,Tanda Progression line,5,Saturday
2024-01-14,7316.4,2284,3.463693,1970-01-01 03:27:49.296433374,21293.7,6797.0,2.661713,319.202393,3.876082,1970-01-01 03:52:33.894558325,...,1.295142e+00,5:12,5:19,0.380245,Latest run,3 hours 53 minutes,circle,Tanda Progression line,6,Sunday
2024-01-15,5060.6,1662,3.677599,1970-01-01 03:40:39.357268770,26354.3,8459.0,3.294287,320.972289,3.882076,1970-01-01 03:52:55.474010320,...,1.473928e+00,5:28,5:20,0.470612,Latest run,3 hours 53 minutes,circle,Tanda Progression line,0,Monday
2024-01-16,5050.0,1826,3.891928,1970-01-01 03:53:30.942333646,31404.3,10285.0,3.925537,327.502922,3.918790,1970-01-01 03:55:07.642943680,...,1.677394e+00,6:01,5:27,0.560791,Latest run,3 hours 55 minutes,circle,Tanda Progression line,1,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-02,8611.6,2877,3.546244,1970-01-01 03:32:46.478312260,286190.2,91625.0,35.773775,320.154219,3.621800,1970-01-01 03:37:18.478698988,...,1.948867e+06,5:34,5:20,5.110539,Latest run,3 hours 37 minutes,circle,Tanda Progression line,1,Tuesday
2024-07-03,6995.7,2221,3.512866,1970-01-01 03:30:46.315957075,288151.8,92077.0,36.018975,319.543380,3.616099,1970-01-01 03:36:57.955909323,...,2.217895e+06,5:17,5:19,5.145568,Latest run,3 hours 37 minutes,circle,Tanda Progression line,2,Wednesday
2024-07-04,6916.7,2225,3.543685,1970-01-01 03:32:37.265364992,291031.6,92737.0,36.378950,318.649246,3.607751,1970-01-01 03:36:27.902687360,...,2.524060e+06,5:21,5:18,5.196993,Latest run,3 hours 36 minutes,circle,Tanda Progression line,3,Thursday


In [26]:
daily_df["day_of_week"] = daily_df.index.strftime("%a")

Index(['Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun',
       ...
       'Wed', 'Thu', 'Fri', 'Sat', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sun'],
      dtype='object', name='date', length=117)

In [15]:
df

,distance_meters,time_seconds
start_date,,
2024-01-12 19:10:16+00:00,5971.8,1920
2024-01-13 14:02:48+00:00,8005.5,2593
2024-01-14 10:15:14+00:00,7316.4,2284
2024-01-15 09:24:27+00:00,5060.6,1662
2024-01-16 07:07:55+00:00,5050.0,1826
...,...,...
2024-07-04 05:51:50+00:00,6916.7,2225
2024-07-05 06:19:27+00:00,5036.6,1568
2024-07-07 11:48:08+00:00,6230.2,2355


In [16]:
pandas.DataFrame(running_activities)

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed
0,2,"{'id': 44717295, 'resource_state': 1}",Evening Run,5971.8,1920,1920,1.0,Run,Run,NaN,...,True,2.8,-0.6,11283572636,11283572636,garmin_ping_314957076441,False,0,0,False
1,2,"{'id': 44717295, 'resource_state': 1}",Afternoon Run,8005.5,2593,3152,26.0,Run,Run,NaN,...,True,4.8,-0.6,11288894877,11288894877,garmin_ping_315099514157,False,0,0,False
2,2,"{'id': 44717295, 'resource_state': 1}",Lunch Run,7316.4,2284,2294,2.0,Run,Run,NaN,...,True,5.0,0.4,11294033151,11294033151,garmin_ping_315247079683,False,0,0,False
3,2,"{'id': 44717295, 'resource_state': 1}",Morning Run,5060.6,1662,1737,35.0,Run,Run,NaN,...,True,4.4,-6.0,11300742903,11300742903,garmin_ping_315427767249,False,0,0,False
4,2,"{'id': 44717295, 'resource_state': 1}",Morning Run,5050.0,1826,1826,0.0,Run,Run,NaN,...,True,0.0,0.0,11306704065,11306704065,garmin_ping_315596550016,False,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2,"{'id': 44717295, 'resource_state': 1}",Morning Run,6916.7,2225,2341,8.0,Run,Run,NaN,...,True,127.2,109.0,12589951788,12589951788,garmin_ping_352416612739,False,0,0,False
122,2,"{'id': 44717295, 'resource_state': 1}",Morning Run,5036.6,1568,1568,2.0,Run,Run,NaN,...,True,126.8,107.8,12598393832,12598393832,garmin_ping_352654071860,False,0,0,False
123,2,"{'id': 44717295, 'resource_state': 1}",Afternoon Run,6230.2,2355,2868,46.0,Run,Run,NaN,...,True,74.4,47.6,12617246087,12617246087,garmin_ping_353188654328,False,0,0,False
124,2,"{'id': 44717295, 'resource_state': 1}",Afternoon Trail Run,2216.5,1549,1701,186.0,Run,TrailRun,NaN,...,True,191.6,52.2,12617247221,12617247221,garmin_ping_353188676112,False,0,0,False


In [17]:
running_activities[0].keys()

dict_keys(['resource_state', 'athlete', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id', 'start_date', 'start_date_local', 'timezone', 'utc_offset', 'location_city', 'location_state', 'location_country', 'achievement_count', 'kudos_count', 'comment_count', 'athlete_count', 'photo_count', 'map', 'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed', 'average_cadence', 'average_watts', 'max_watts', 'weighted_average_watts', 'kilojoules', 'device_watts', 'has_heartrate', 'average_heartrate', 'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed'])

In [18]:
cols = [
    # "resource_state",
    # "athlete",
    "name",
    "distance",
    "moving_time",
    # "elapsed_time",
    # "total_elevation_gain",
    # "type",
    "sport_type",
    # "workout_type",
    # "id",
    # "start_date",
    "start_date_local",
    # "timezone",
    # "utc_offset",
    # "location_city",
    # "location_state",
    # "location_country",
    # "achievement_count",
    # "kudos_count",
    # "comment_count",
    # "athlete_count",
    # "photo_count",
    # "map",
    # "trainer",
    # "commute",
    # "manual",
    # "private",
    # "visibility",
    # "flagged",
    # "gear_id",
    # "start_latlng",
    # "end_latlng",
    "average_speed",
    # "max_speed",
    # "average_cadence",
    # "average_watts",
    # "max_watts",
    # "weighted_average_watts",
    # "kilojoules",
    # "device_watts",
    # "has_heartrate",
    "average_heartrate",
    "max_heartrate",
    # "heartrate_opt_out",
    # "display_hide_heartrate_option",
    # "elev_high",
    # "elev_low",
    # "upload_id",
    # "upload_id_str",
    # "external_id",
    # "from_accepted_tag",
    # "pr_count",
    # "total_photo_count",
    # "has_kudoed",
]
pandas.DataFrame(running_activities)[cols]

,name,distance,moving_time,sport_type,start_date_local,average_speed,average_heartrate,max_heartrate
0,Evening Run,5971.8,1920,Run,2024-01-12T20:10:16Z,3.110,167.2,182.0
1,Afternoon Run,8005.5,2593,Run,2024-01-13T15:02:48Z,3.087,152.2,181.0
2,Lunch Run,7316.4,2284,Run,2024-01-14T11:15:14Z,3.203,152.3,170.0
3,Morning Run,5060.6,1662,Run,2024-01-15T10:24:27Z,3.045,149.6,172.0
4,Morning Run,5050.0,1826,Run,2024-01-16T08:07:55Z,2.766,131.5,145.0
...,...,...,...,...,...,...,...,...
121,Morning Run,6916.7,2225,Run,2024-07-04T07:51:50Z,3.109,154.8,165.0
122,Morning Run,5036.6,1568,Run,2024-07-05T08:19:27Z,3.212,156.8,176.0
123,Afternoon Run,6230.2,2355,Run,2024-07-07T13:48:08Z,2.646,151.2,168.0
124,Afternoon Trail Run,2216.5,1549,TrailRun,2024-07-07T14:37:04Z,1.431,144.4,176.0


In [19]:
pandas.DataFrame(running_activities)[cols].to_csv(
    "../static/dummy/running_activities.csv", index=False
)